In [ ]:
# Import necessary libraries
import os
import re
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from zipfile import ZipFile

In [ ]:
# Define the names of the directories to be created
data_folder = 'data'
output_folder = 'output'

# Check if the directory exists; if not, create it
if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
# Function to download data
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)


# Call the download function with the specified URL
download('https://github.com/spatialthoughts/python-tutorials/raw/main/data/' +
         'Hurricane_Evacuation_Centers.xlsx')

In [ ]:
# Load hurricane evacuation center data
excel_file = 'Hurricane_Evacuation_Centers.xlsx'
excel_file_path = os.path.join(data_folder, excel_file)
address_df = pd.read_excel(excel_file_path, skiprows=[0])
address_df

In [ ]:
def make_ordinal(match):
    n = int(match.group(1))
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix + match.group(2)

def update_address(row):
  old_address = row['ADDRESS']
  pattern = r'(\d+)(\s+(?:Street|Avenue|Blvd|Drive))'
  result = re.sub(pattern, make_ordinal, old_address)
  return result

address_df['ADDRESS_FIXED'] = address_df.apply(update_address, axis=1)
address_df

In [ ]:
address_df['Full_Address'] = (
    address_df['ADDRESS_FIXED'] + ',' +
    'NYC' + ',' +
    address_df['STATE']+ ',' +
    address_df['ZIP_CODE'].astype(str))
address_df

In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas()

locator = Nominatim(user_agent='spatialthoughts', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

address_df_pd = address_df.copy()
address_df_pd['location'] = address_df_pd['Full_Address'].progress_apply(geocode)
address_df_pd

In [ ]:
address_df_pd['latitude'] = address_df_pd['location'].apply(lambda loc: loc.latitude if loc else None)
address_df_pd['longitude'] = address_df_pd['location'].apply(lambda loc: loc.longitude if loc else None)
address_df_pd

In [ ]:
failed = address_df_pd[address_df_pd['location'].isna()]
failed

In [ ]:
address_df_pd.loc[54, ['latitude', 'longitude']] = (40.85339,-73.93350)

In [ ]:
address_df_pd = address_df_pd[['EC_Name', 'Full_Address', 'latitude', 'longitude']]
address_df_pd.rename(columns = {'EC_Name': 'Name', 'Full_Address': 'Address'}, inplace=True)

In [ ]:
geometry = gpd.points_from_xy(address_df_pd.longitude, address_df_pd.latitude)
address_gdf = gpd.GeoDataFrame(address_df_pd, crs='EPSG:4326', geometry=geometry)
address_gdf

In [ ]:
import folium
m = leafmap.Map(width=800, height=500)
address_gdf.explore(
    m=m,
    marker_type='marker',
    marker_kwds={
        'icon': folium.Icon(color='#fdbb84', icon='hurricane', prefix='fa')
    }
)
m.zoom_to_gdf(address_gdf)
m

In [ ]:
output_file = 'hurricane_evacuation_centers.shp'
output_path = os.path.join(output_folder, output_file)

address_gdf.to_file(filename=output_path)

In [ ]:
output_zip_file = 'hurricane_evacuation_centers.zip'
output_zip_path = os.path.join(output_folder, output_zip_file)

sidecar_files = [
    os.path.join(output_folder, file)
    for file in os.listdir(output_folder)
    if file.endswith(('shp', 'shx', 'dbf', 'prj'))
]

with ZipFile(output_zip_path, 'w') as zip_object:
    for sidecar in sidecar_files:
      zip_object.write(sidecar, os.path.basename(sidecar))